In [2]:
import json
import os
import sys
from pprint import pprint
# Example usage
gt_path_1 = "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level1.jsonl"
gt_path_2 = "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level2.jsonl"
gt_path_3 = "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level3.jsonl"
query_path_1 = "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level1_queries.json"
query_path_2 = "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level2_queries.json"
query_path_3 = "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level3_queries.json"

def load_dataset(file_path):
    """
    Load the dataset from a JSON file.
    """
    with open(file_path, 'r') as f:
        return json.load(f)

def read_json_to_dict(filename):
    api_info = []
    with open(filename, 'r') as file:
        for line in file:
            api_info.append(json.loads(line))

    return api_info
import json

gt = read_json_to_dict(gt_path_1)
q = load_dataset(query_path_1)
gt = sorted(gt, key=lambda x: x['Id'])
q = sorted(q, key=lambda x: x['Index'])
pprint(len(gt))
pprint(len(q))
pprint(gt[0])
pprint(q[0])

100
100
{'Id': 42,
 'gold_data': {'conversations': [{'value': '[{"api": "selectoutfit", '
                                           '"parameters": {"weather": "{{ '
                                           'tasks.checkweather.result }}", '
                                           '"occasion": "{{ '
                                           'inputs.parameters.occasion }}"}}, '
                                           '{"api": "checkweather", '
                                           '"parameters": {"location": "{{ '
                                           'inputs.parameters.location '
                                           '}}"}}]'}],
               'topology': ['selectoutfit', 'checkweather']}}
{'Index': 42,
 'Query': 'Could you help me select what to wear for a date in Chicago '
          'tonight?'}


In [20]:
import sys
sys.path.append('/home/UNT/ae0589/project/action_engine/utils/')

from llms import call_llm, get_model
from utilities import escape_json
from schemas.workflow import Tasks

def subtask_diviser(model_name, user_query, functions):
    SYSTEM_PROMPT = f'''
    Instruction: 
    User Request: {user_query}
    Selected Functions: {functions}

    Given a user request and APIs to use. Your goal is to divide the user request into subtasks if the user request is a compound workload and extract information.
    Let's think step by step.
    Each subtask will be converted to an API in the next step. Please divide the given user request into the one smallest executable subtasks.
    Do not include the given funciton name in the subtask, and the number of subtask must be the same number with given function..
    Below are examples of user requests and the divided subtasks:

    Example 1:
    User Request: "Can you add the song 'Imagine' by John Lennon to Jenny's 'Chill Vibes' playlist?" 
    APIs: ["UserName2ID", "PlaylistName2ID", "AddSongToPlaylist"]
    {{'Tasks': [
            {{
                "subtask_number": 1, 
                "subtask_description": "Convert the username 'Jenny' to a user ID."
            }}, 
            {{
                "subtask_number": 2, 
                "subtask_description": "Convert the playlist name 'Chill Vibes' to a playlist."
            }}, 
            {{
                "subtask_number": 3, 
                "subtask_description": "Add the song 'Imagine' by John Lennon to the playlist."
            }}
        ]
    }}
    Example 2:
    User Request: "Can you help me make a reservation in a 3-star hotel in Chicago from July 20 to July 25?"
    APIs: ["FindHotel", "BookRoom"]
    {{'Tasks': [
            {{
                "subtask_number": 1, 
                "subtask_description": "Find a 3-star hotel in Chicago using the FindHotel API."
            }}, 
            {{
                "subtask_number": 2, 
                "subtask_description": "Make a reservation in the hotel found with the check-in date 'July 20' and check-out date 'July 25'."
            }}
        ]
    }}
    You must only generate the Answer. Your answer must be in JSON format. 
    Your Answer format must start with "```json" and end with "```".
    '''

    USER_PROMPT = f"""
    Answer: 
    """
    response = call_llm(model_name, Tasks, "Tasks" ,escape_json(SYSTEM_PROMPT), escape_json(USER_PROMPT))

    return response["Tasks"]

model_name = "gpt-4o"
model = get_model(model_name)

In [21]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
filepath = "/home/UNT/ae0589/project/action_engine/db/api_info/"
filename = filepath + 'api_information.json'
NO_FUNC = False
api_info = load_dataset(filename)

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = openai_api_key

embedding_function = OpenAIEmbeddings(model="text-embedding-3-large")
loaded_faiss = FAISS.load_local(filepath + 'vectordb/LangChain_FAISS/', embedding_function, "api_vec", allow_dangerous_deserialization=True)

def get_function_info(function_name, api_info):
    """
    Retrieve detailed information about a function by its name, case-insensitively.

    Args:
    - function_name (str): The name of the function to search for.
    - api_info (list): List of function metadata dictionaries.

    Returns:
    - dict: The metadata for the specified function, or None if not found.
    """
    # Normalize the input function name to lowercase
    normalized_function_name = function_name.strip().lower()

    for function in api_info:
        # Normalize the stored function name to lowercase for comparison
        if function.get("name", "").strip().lower() == normalized_function_name:
            return function
        
    return None

In [22]:
gt = read_json_to_dict(gt_path_3)
q = load_dataset(query_path_3)
gt = sorted(gt, key=lambda x: x['Id'])
q = sorted(q, key=lambda x: x['Index'])

# Target Index to search for
target_index = 887

# Find the specific dictionary using a list comprehension
specific_gt = next((item for item in gt if item['Id'] == target_index), None)
specific_result = next((item for item in q if item['Index'] == target_index), None)

# Output the result
if specific_result:
    # pprint(specific_gt)
    # pprint(specific_result)
    pass
else:
    print(f"No item found with Index {target_index}")

result = []
selected_functions = []
user_query = specific_result["Query"]
functions = specific_gt["gold_data"]["topology"][::-1]
subtasks = subtask_diviser(model, user_query, functions)
print(functions)
# print(subtasks)
for j in range(len(functions)):
    func_info_list = get_function_info(functions[j], api_info)
    func_info_list["task_num"] = subtasks[j]["subtask_number"]
    func_info_list["task_description"]= subtasks[j]["subtask_description"]
    selected_functions.append(func_info_list)
result.append(
    {"Id": specific_result["Index"], "Selected_Functions": selected_functions}
)
pprint(result)

# Save results to file
with open("/home/UNT/ae0589/project/action_engine/eval/data/test_query/param_evaluation/add.json", "w") as f:
    json.dump(result, f, indent=4)

Attempt 1/3 - Invoking LLM.
Extraction successful.
['findrestaurants', 'getweatherforecast', 'plandayout', 'findoutdooractivities']
[{'Id': 887,
  'Selected_Functions': [{'Id': 310,
                          'description': 'This API is to find restaurants of a '
                                         'specific cuisine type in a specific '
                                         'city.',
                          'format': 'FindRestaurants(city, cuisineType) -> '
                                    'restaurants',
                          'input_parameters_with_datatype': [{'datatype': 'String',
                                                              'description': 'the '
                                                                             'name '
                                                                             'of '
                                                                             'the '
                                                           

In [96]:
import json
import traceback

# Define paths
gt_paths = [
    "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level1.jsonl",
    "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level2.jsonl",
    "/home/UNT/ae0589/project/action_engine/eval/data/gt/gt_level3.jsonl",
]
query_paths = [
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level1_queries.json",
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level2_queries.json",
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/level3_queries.json",
]
output_files = [
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/param_evaluation/result_level1.json",
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/param_evaluation/result_level2.json",
    "/home/UNT/ae0589/project/action_engine/eval/data/test_query/param_evaluation/result_level3.json",
]

error_log_file = "/home/UNT/ae0589/project/action_engine/eval/data/test_query/param_evaluation/error_log.txt"

# Function to log errors
def log_error(log_file, message):
    with open(log_file, "a") as f:
        f.write(message + "\n")

# Process each pair of gt and query paths
for level, (gt_path, query_path, output_path) in enumerate(zip(gt_paths, query_paths, output_files), start=1):
    try:
        print(f"Processing level {level}...")

        # Load and sort data
        gt = read_json_to_dict(gt_path)
        q = load_dataset(query_path)
        gt = sorted(gt, key=lambda x: x['Id'])
        q = sorted(q, key=lambda x: x['Index'])

        result = []
        for i in range(len(q)):
            try:
                selected_functions = []
                user_query = q[i]["Query"]
                functions = gt[i]["gold_data"]["topology"][::-1]
                subtasks = subtask_diviser(model, user_query, functions)  # Adjust this function as needed
                for j in range(len(functions)):
                    func_info_list = get_function_info(functions[j], api_info)
                    func_info_list["task_num"] = subtasks[j]["subtask_number"]
                    func_info_list["task_description"] = subtasks[j]["subtask_description"]
                    selected_functions.append(func_info_list)
                result.append(
                    {"Id": q[i]["Index"], "Selected_Functions": selected_functions}
                )
            except Exception as e:
                # Log specific query-level failure
                error_message = (
                    f"Error processing query at level {level}, Query ID: {q[i]['Index']}\n"
                    f"Error: {str(e)}\n"
                    f"{traceback.format_exc()}\n"
                )
                log_error(error_log_file, error_message)

        # Save results to file
        with open(output_path, "w") as f:
            json.dump(result, f, indent=4)

        print(f"Results saved for level {level} to {output_path}")

    except Exception as e:
        # Log level-wide failure
        error_message = (
            f"Error processing level {level} files.\n"
            f"GT Path: {gt_path}, Query Path: {query_path}\n"
            f"Error: {str(e)}\n"
            f"{traceback.format_exc()}\n"
        )
        log_error(error_log_file, error_message)

Processing level 1...
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1/3 - Invoking LLM.
Extraction successful.
Attempt 1